In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import os
import json

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [2]:

def clean_text(string: str, 
               punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
               stop_words = stopwords.words('english'),
               # porter = PorterStemmer()
               wnl = WordNetLemmatizer()
              ):
    """
    A method to clean text. It removes punctuations, stop words, applies lemmatization.
    """
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # stemming/lemmatizing words. That means changing word to its basic format, for example
    # words 'fishing', 'fished', 'fischer' will be changed into a word 'fisch'
    # lemmatization should be better because stemming changes words too much, for example
    # business is changed into busi
    # string = ' '.join([porter.stem(word) for word in string.split()])
    string = ' '.join([wnl.lemmatize(word, pos = "v") for word in string.split()])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string

def create_training_data(tokenizer,
                         max_sen_len,
                         sentences_file,
                         embed_matrix_file,
                         model_folder
                        ):
    """
    Creating a training and testing datasets self.x_train, self.x_test, self.y_train, self.y_test. This function
    also creates and saves a tokenizer and a list of all unique tables names all_unique_values because when we load
    a ready model those values are needed for the 'predict' function.
    """
    sentences_tables = pd.read_excel(sentences_file).values
    random.shuffle(sentences_tables)
    clean_sentences = np.array([clean_text(sentence) for sentence in sentences_tables[:, 0]])

    tokenizer.fit_on_texts(clean_sentences)

    sequences = tokenizer.texts_to_sequences(clean_sentences)
    x = pad_sequences(sequences, maxlen = max_sen_len)

    embed_matrix = pd.read_csv(embed_matrix_file).values

    x_train, x_test = train_test_split(x, test_size = 0.2)

    with open(os.path.join(model_folder, 'tokenizer.json'), 'w') as file:
        json.dump(tokenizer.to_json(), file)
        
    return x_train, x_test


def get_coefs(word, *arr): 
    return word, list(np.asarray(arr, dtype='float'))


def create_embedding_file(tokenizer,
                          embed_file_src = r'model\glove.840B.300d.txt', 
                          embed_file_trg = r'model\model_embeddings.txt'
                         ):
    """
    This function will create an embedding file called embed_file_trg which will contain only those words 
    from embed_file_src which are present in the training dataset. If training dataset wasn't created yet
    then this function will create it.
    """
    # creating a training dataset if we didn't do that yet
    # if not hasattr(self, 'tokenizer'):
    #     self.create_training_data()

    embeddings = dict(get_coefs(*o.split(" ")) for o in open(embed_file_src, errors = 'ignore'))
    with open(embed_file_trg, 'w') as file:
        for word, index in tokenizer.word_index.items():
            word_vector = embeddings[word]
            line = ' '.join(np.concatenate([[word], word_vector]))
            file.write(line + '\n')


def create_embedding_matrix(tokenizer,
                            model_folder,
                            word_vec_dim,
                            embed_file_path,
                           ):
    """
    A function to create the embedding matrix. This is a matrix where each row is a vector representing a word.
    To create that matrix we use a word embedding file which path is equal to embedding_file_path.
    embedding_matrix[row_number] is a vector representation for a word = list(tokenizer.word_index.keys())[row_number - 1]
    First row of embedding_matrix are zeros. This matrix is needed to train a model.
    """
    embeddings = dict(get_coefs(*o.split(" ")) for o in open(embed_file_path, errors = 'ignore'))

    # embedding_matrix[row_number] is a vector representation of a word = self.tokenizer.word_index.keys()[row_number - 1]
    # first row in embedding_matrix is 0
    embedding_matrix = np.zeros((len(tokenizer.word_counts) + 1, word_vec_dim))
    for word, index in tokenizer.word_index.items():
        if index > len(tokenizer.word_counts):
            break
        else:
            try:
                embedding_matrix[index] = embeddings[word]
            except:
                continue

    pd.DataFrame(embedding_matrix).to_csv(os.path.join(model_folder, 'embedding_matrix.csv'))
    return embedding_matrix

In [3]:
tokenizer = Tokenizer()
max_sen_len = 20
sentences_file = r'data\sentences_tables.xlsx'
embed_matrix_file = r'model\embedding_matrix.csv'
model_folder = 'model'
word_vec_dim = 300
embed_file_path = r'model\model_embeddings.txt'

In [4]:
x_train, x_test = create_training_data(
    tokenizer = tokenizer, 
    max_sen_len = max_sen_len,
    sentences_file = sentences_file,
    embed_matrix_file = embed_matrix_file,
    model_folder = model_folder
)

In [5]:
x_train

array([[ 0,  0,  0, ..., 14, 26, 16],
       [ 0,  0,  0, ...,  0, 11, 15],
       [ 0,  0,  0, ..., 10,  2, 27],
       ...,
       [ 0,  0,  0, ...,  1, 13, 12],
       [ 0,  0,  0, ..., 18, 23, 13],
       [ 0,  0,  0, ..., 22, 37, 41]])

In [6]:
embed_matrix = create_embedding_matrix(
    tokenizer = tokenizer,
    model_folder = model_folder,
    word_vec_dim = word_vec_dim,
    embed_file_path = embed_file_path
)

In [7]:
embed_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.50318 ,  0.27905 , -0.045497, ...,  0.4781  ,  0.13005 ,
        -0.014399],
       [ 0.16082 ,  0.11008 , -0.28129 , ...,  0.12435 , -0.27433 ,
         0.65513 ],
       ...,
       [ 0.37492 , -0.052425, -0.60094 , ..., -0.36104 , -0.065253,
        -0.1206  ],
       [ 0.012832,  0.22669 , -0.17511 , ...,  0.17134 ,  0.040047,
        -0.37131 ],
       [-0.39054 , -0.55117 , -0.073466, ...,  0.34569 ,  0.30918 ,
        -0.32873 ]])

In [38]:
class Encoder(Model):
    def __init__(self,
                 vocab_size,
                 embedding_dim,
                 units,
                 batch_size,
                 embed_matrix
                ):
        super().__init__()
        self.units = units
        self.batch_size = batch_size
        self.embedding = layers.Embedding(
            input_dim = embed_matrix.shape[0],
            output_dim = embedding_dim,
            embeddings_initializer = tf.keras.initializers.Constant(embed_matrix),
            trainable = False
        )
        self.lstm = layers.LSTM(
            units = self.units,
            return_sequences = True,
            return_state = True
        )
        
    
    def call(self, x, state_h, state_c):
        # make sure that the types are correct
        x = tf.cast(x, tf.float32)
        state_h = tf.cast(state_h, tf.float32)
        state_c = tf.cast(state_c, tf.float32)
        # x.shape = (batch_size, max_sen_len)
        # x is a series of numbers which represent words
        # state_h.shape = (batch_size, state_size)
        x = self.embedding(x)
        # x = tf.reshape(x, [1, x.shape[0], x.shape[1]])
        output, state_h, state_c = self.lstm(x, initial_state = [state_h, state_c])
        return output, state_h, state_c
    
    def initialize_hidden_state(self):
        state_h = tf.zeros((self.batch_size, self.units))
        state_c = tf.zeros((self.batch_size, self.units))
        return state_h, state_c

In [9]:
# version 1 like in Jonathan Hui pdf
class Bahdau_attention(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.W1 = layers.Dense(units)
        self.W2 = layers.Dense(units)
        self.V = layers.Dense(1)
        
    def call(self, decoder_hidden, encoder_hidden):
        # decoder_hidden.shape = (batch_size, hidden_size)
        # decoder_hidden_time_axis.shape = (batch_size, 1, hidden_size)
        decoder_hidden_time_axis = tf.expand_dims(decoder_hidden, 1)
        
        # encoder_hidden.shape = (batch_size, max_sen_len, hidden_size)
        # argument for tanh shape = (batch_size, max_sen_len, hidden_size)
        # score.shape = (batch_size, max_sen_len, 1)
        score = self.V(tf.nn.tanh(self.W1(decoder_hidden_time_axis) + self.W2(encoder_hidden)))
        
        # attention_weights.shape = (batch_size, max_sen_len, 1)
        attention_weights = tf.nn.softmax(score, axis = 1)
        
        # context_vector.shape = (batch_size, hidden_size)
        context_vector = attention_weights * encoder_hidden
        context_vector = tf.reduce_sum(context_vector, axis = 1)
        
        return context_vector, attention_weights

In [59]:
# version 2, like in 'attention explanation.pdf'
class Bahdau_attention(layers.Layer):
    def __init__(self):
        super().__init__()
        self.dense = layers.Dense(1)
        
    def call(self, decoder_state_h, encoder_states_h):
        # decoder_state_h.shape = (batch_size, hidden_size)
        # encoder_states_h.shape = (batch_size, max_sen_len, hidden_size)
        # encoder_states_h_flattened.shape = (max_sen_len, hidden_size)
        encoder_states_h_flattened = tf.reshape(encoder_states_h, [-1, tf.shape(encoder_states_h)[2]])
        try:
            batch_size, hidden_size = tf.gather(tf.shape(decoder_state_h), [0,1])
        except:
            print(decoder_state_h)
        # context_vector.shape = (batch_size, hidden_size)
        context_vector = np.zeros([batch_size, hidden_size])
        for i in range(batch_size):
            for j in range(max_sen_len):
                e = []
                for k in range(max_sen_len):
                    x = tf.concat([decoder_state_h[i], encoder_states_h_flattened[i + k]], 0)
                    x = tf.reshape(x, [1, tf.shape(x)[0]])
                    e.append(tf.math.exp(self.dense(x)))
                e = tf.stack(e)

                attention_weight = e[j] / tf.math.reduce_sum(e)
                context_vector[i] += attention_weight * encoder_states_h_flattened[i + j]
        
        return tf.cast(tf.stack(context_vector), tf.float32)

In [82]:
class Decoder(Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size, embed_matrix):
        super().__init__()
        self.units = units
        self.batch_size = batch_size
        self.embedding = layers.Embedding(
            input_dim = embed_matrix.shape[0],
            output_dim = embedding_dim,
            embeddings_initializer = tf.keras.initializers.Constant(embed_matrix),
            trainable = False
        )
        self.lstm = layers.LSTM(
            units = self.units,
            return_sequences = True,
            return_state = True
        )
        self.dense = layers.Dense(vocab_size)
        self.attention = Bahdau_attention()
        
    def call(self, x, decoder_state_h, decoder_state_c, encoder_states_h):
        # x.shape = (batch_size, 1)
        # x is a single number for each batch representing a single word
        # encoder_states_h.shape = (batch_size, max_sen_len, hidden_size)
        # decoder_state_h.shape = (batch_size, 1, hidden_size)
        
        # make sure that the types are correct
        x = tf.cast(x, tf.float32)
        decoder_state_h = tf.cast(decoder_state_h, tf.float32)
        decoder_state_c = tf.cast(decoder_state_c, tf.float32)
        encoder_states_h = tf.cast(encoder_states_h, tf.float32)
        
        # context_vector.shape = (batch_size, hidden_size)
        context_vector = self.attention(decoder_state_h, encoder_states_h)
        # shape of output of embedding layer = (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x.shape after concatenation = (batch_size, 1, hidden_size + embedding_dim)
        try:
            x = tf.concat([tf.expand_dims(context_vector, 1), x], axis = 2)
        except Exception as e:
            print(context_vector)
            print(x)
            print(e)
            return
        
        # output, state_h, state_c = self.lstm(x)
        # output, state_h, state_c = self.lstm(x, initial_state = [tf.reshape(decoder_state_h, [decoder_state_h.shape[0], decoder_state_h.shape[2]]),
        #                                                          tf.reshape(decoder_state_c, [decoder_state_c.shape[0], decoder_state_c.shape[2]])
        #                                                         ])
        output, state_h, state_c = self.lstm(x, initial_state = [decoder_state_h, decoder_state_c])
        
        output = tf.reshape(output, [-1, tf.shape(output)[2]])
        # output.shape = (batch_size, vocab_size)
        output = self.dense(output)
        
        return output, state_h, state_c

In [93]:
def train_step(inp, 
               targ, 
               enc_state_h, 
               enc_state_c, 
               batch_size, 
               encoder, 
               decoder, 
               loss_function, 
               optimizer):
    # inp.shape = targ.shape (batch_size, max_sen_len, 1)
    # enc_state_h.shape = (batch_size, state_size)
    
    # make sure that the types are correct
    inp = tf.cast(inp, tf.float32)
    targ = tf.cast(targ, tf.float32)
    enc_state_h = tf.cast(enc_state_h, tf.float32)
    enc_state_c = tf.cast(enc_state_c, tf.float32)
    
    batch_loss = 0
    
    with tf.GradientTape() as tape:
        # enc_output.shape = (batch_size, max_sen_len, state_size)
        # enc_state_h.shape = (batch_size, state_size)
        enc_output, enc_state_h, enc_state_c = encoder(inp, enc_state_h, enc_state_c)
        dec_state_h = enc_state_h
        dec_state_c = enc_state_c
        
        # dec_input.shape = (batch_size, 1)
        dec_input = tf.expand_dims([0] * batch_size, 1)
        
        for t in range(len(targ)):
            prediction, dec_state_h, dec_state_c, = decoder(dec_input, dec_state_h, dec_state_c, enc_output)
            batch_loss += loss_function(targ[:, t], prediction)
            dec_input = tf.expand_dims(targ[:, t], 1)
            
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(batch_loss, variables)
    optimizer.apply_gradients(grads_and_vars = zip(gradients, variables))
    
    return batch_loss

In [98]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype = loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [99]:
max_sen_len = 3
batch_size = 2

inp = tf.constant([[1,2,3], [4,5,6]])
targ = tf.constant([[1,2,3], [4,5,6]])
enc_state_h = tf.constant([[1,2,3], [4,5,6]])
enc_state_c = tf.constant([[1,2,3], [4,5,6]])

decoder = Decoder(vocab_size = 1000,
                  embedding_dim = 300,
                  units = 3,
                  batch_size = 2,
                  embed_matrix = embed_matrix
                 )

encoder = Encoder(vocab_size = 1000,
                  embedding_dim = 300,
                  units = 3,
                  batch_size = 2,
                  embed_matrix = embed_matrix
                 )

In [129]:
train_step(inp = inp, 
           targ = targ, 
           enc_state_h = enc_state_h, 
           enc_state_c = enc_state_c, 
           batch_size = 2,
           encoder = encoder, 
           decoder = decoder, 
           loss_function = loss_function, 
           optimizer = optimizer
          )

<tf.Tensor: shape=(), dtype=float32, numpy=13.36879>